In [1]:
# Init
import glob
import os
import pickle
import sys

module_path = os.path.abspath(os.path.join("../src/simulicronalpha/"))
if module_path not in sys.path:
    sys.path.append(module_path)

import multiprocessing
import warnings

# Imports
import numpy as np
import pandas as pd

np.set_printoptions(suppress=True)
# For loading data
import concurrent.futures
import gc
import json
import random

# Plotting imports
import altair as alt
import seaborn as sns
from matplotlib.colors import ListedColormap
from numpy import concatenate as c
from numpy import cumsum

In [2]:
# Acc. functions
def returnLast(lst):
    return lst[-1]


def colorRanges(i):
    # Construct the colormap
    current_palette = sns.color_palette("muted", n_colors=5)
    cmap = ListedColormap(sns.color_palette(current_palette).as_hex())
    # range and stuff
    if i == 0:
        return "0"
    elif i > 0 and i <= 20:
        return "1-20"
    elif i > 20 and i <= 50:
        return "20-50"
    elif i > 50 and i <= 100:
        return "50-100"
    elif i > 100 and i <= 200:
        return "100-200"
    elif i > 200 and i <= 300:
        return "200-300"
    elif i > 300:
        return ">300"


# Since pickle format is not working as intended.
# Insert angry thoughts abouts pickle format here
# A function to open pickle files and convert into json
def pickleToJson(resultDir):
    for file in glob.glob(os.path.join(resultDir, "*.pickle")):
        with open(file, "rb") as f:
            gc.disable()
            data = pickle.load(f)
            del data["TEset"]
            gc.enable()
            with open(file + ".json", "w") as fp:
                json.dump(data, fp)
    return 0

In [3]:
current_palette = sns.color_palette("muted", n_colors=5)
cmap = ListedColormap(sns.color_palette(current_palette).as_hex())

In [4]:
# Path to results
resultDir = "/home/siddharth/git/Simulicron/Project_1C/etaVsEpi/"

# Convert them to json
returnValue = pickleToJson(resultDir)

In [5]:
# Read the json files into pandas dataframe
SimulationRecords = []
for filename in glob.glob(os.path.join(resultDir, "*.pickle.json")):
    with open(filename, 'r') as file:
        SimulationRecords.append(json.load(file))

SimulationResultsDataFrame = pd.DataFrame(SimulationRecords)

In [6]:
SimulationResultsDataFrame

,State,Generatrion,NTE,AvgCopyNum,CopyNumVar,TEfamilyCN,TEfamilyVR,TEfamilyRg,AvgFit,HGTGen,...,NTI,Freq,TRate,Tau,selPen,piRNA,TEpi,FitnessFunction,epistasisCoefficient,TECoreOverlap
0,FLUX,2501,489026,"[0.2, 0.412, 0.851, 1.487, 2.846, 5.45, 9.57, ...","[0.394, 1.3902560000000002, 3.358799, 6.873831...","{'1': [0.2, 0.412, 0.851, 1.487, 2.846, 5.45, ...","{'1': [[0.394], [1.3902560000000002], [3.35879...","{'1': [0.0, 0.005, 0.015, 0.038, 0.079, 0.16, ...","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,2,"[0.2, 0.2]","[1.0, 0.48849423517857793]",1.0,0.0,"[3, 4, 5, 6, 7, 42, 43, 44, 45, 46, 322, 323, ...","{'1': 5.62, '2': 5.566}",2,1.748802,"[0, {'1': 0.005, '2': 0.0}, {'1': 0.016, '2': ..."
1,FLUX,2501,363153,"[0.2, 0.416, 0.877, 1.607, 2.948, 5.466, 10.19...","[0.36199999999999993, 1.226944, 3.333871000000...","{'1': [0.2, 0.416, 0.877, 1.607, 2.948, 5.466,...","{'1': [[0.36199999999999993], [1.226944], [3.3...","{'1': [0.0, 0.005, 0.015, 0.038, 0.077, 0.152,...","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,2,"[0.2, 0.2]","[1.0, 0.7727469940922058]",1.0,0.0,"[153, 154, 155, 156, 157, 192, 193, 194, 195, ...","{'1': 6.387, '2': 7.286}",2,-0.202203,"[0, {'1': 0.005, '2': 0.0}, {'1': 0.012, '2': ..."
2,FLUX,2501,454206,"[0.2, 0.368, 0.752, 1.407, 2.642, 4.725, 8.69,...","[0.3499999999999999, 1.0085760000000001, 2.620...","{'1': [0.2, 0.368, 0.752, 1.407, 2.642, 4.725,...","{'1': [[0.3499999999999999], [1.00857600000000...","{'1': [0.0, 0.004, 0.027, 0.046, 0.069, 0.129,...","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,2,"[0.2, 0.2]","[1.0, 0.21676181427146657]",1.0,0.0,"[108, 109, 110, 111, 112, 178, 179, 180, 181, ...","{'1': 8.686, '2': 4.584}",2,-1.120283,"[0, {'1': 0.006, '2': 0.0}, {'1': 0.024, '2': ..."
3,FLUX,2501,379290,"[0.2, 0.43, 0.854, 1.618, 3.336, 6.24, 10.709,...","[0.3279999999999999, 1.1551, 3.056683999999999...","{'1': [0.2, 0.43, 0.854, 1.618, 3.336, 6.24, 1...","{'1': [[0.3279999999999999], [1.1551], [3.0566...","{'1': [0.0, 0.004, 0.015, 0.052, 0.092, 0.176,...","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,2,"[0.2, 0.2]","[1.0, 0.12362589143831948]",1.0,0.0,"[99, 100, 101, 102, 103, 140, 141, 142, 143, 1...","{'1': 7.808, '2': 4.729}",2,0.696495,"[0, {'1': 0.009, '2': 0.0}, {'1': 0.019, '2': ..."
4,FLUX,2501,528252,"[0.2, 0.396, 0.795, 1.581, 3.14, 5.664, 9.681,...","[0.346, 1.0511840000000001, 2.8329749999999994...","{'1': [0.2, 0.396, 0.795, 1.581, 3.14, 5.664, ...","{'1': [[0.346], [1.0511840000000001], [2.83297...","{'1': [0.0, 0.004, 0.012, 0.028, 0.084, 0.179,...","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,2,"[0.2, 0.2]","[1.0, 0.5040161716396165]",1.0,0.0,"[32, 33, 34, 35, 36, 109, 110, 111, 112, 113, ...","{'1': 8.916, '2': 5.443}",2,-1.614809,"[0, {'1': 0.003, '2': 0.0}, {'1': 0.014, '2': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,FLUX,2501,408587,"[0.2, 0.454, 0.93, 1.836, 3.43, 6.358, 10.868,...","[0.35, 1.333884, 3.7091000000000003, 8.937104,...","{'1': [0.2, 0.454, 0.93, 1.836, 3.43, 6.358, 1...","{'1': [[0.35], [1.333884], [3.7091000000000003...","{'1': [0.0, 0.008, 0.023, 0.038, 0.09, 0.19, 0...","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,2,"[0.2, 0.2]","[1.0, 0.23068734245909872]",1.0,0.0,"[142, 143, 144, 145, 146, 156, 157, 158, 159, ...","{'1': 11.92, '2': 2.831}",2,1.798857,"[0, {'1': 0.007, '2': 0.0}, {'1': 0.021, '2': ..."
4996,FLUX,2501,337829,"[0.2, 0.36, 0.7, 1.291, 2.353, 4.349, 8.282, 1...","[0.3739999999999999, 1.0384, 2.512, 5.92631899...","{'1': [0.2, 0.36, 0.7, 1.291, 2.353, 4.349, 8....","{'1': [[0.3739999999999999], [1.0384], [2.512]...","{'1': [0.0, 0.005, 0.012, 0.031, 0.062, 0.125,...","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,2,"[0.2, 0.2]","[1.0, 0.20219891305853366]",1.0,0.0,"[56, 57, 58, 59, 60, 148, 149, 150, 151, 152, ...","{'1': 7.386, '2': 0.0}",2,-0.008331,"[0, {'1': 0.004, '2': 0.0}, {'1': 0.018, '2': ..."

In [7]:
# Create a copy for inplace handling
resultDataframeC = SimulationResultsDataFrame.copy()

In [8]:
# Perform some data wrangling
resultDataframeC[["TranspositionRate_1", "TranspositionRate_2"]] = pd.DataFrame(
    resultDataframeC.TRate.tolist(), index=resultDataframeC.index
)
resultDataframeC[["TE_1_CN", "TE_2_CN"]] = resultDataframeC["TEfamilyCN"].apply(
    pd.Series
)
resultDataframeC[["TE_1_RG", "TE_2_RG"]] = resultDataframeC["TEfamilyRg"].apply(
    pd.Series
)
resultDataframeC[["TE_1_pi", "TE_2_pi"]] = resultDataframeC["TEpi"].apply(
    pd.Series
)

# Add column for the last acn (average copy number) for each transposon family
resultDataframeC["TE_1_ACN"] = resultDataframeC["TE_1_CN"].apply(
    lambda x: returnLast(x)
)
resultDataframeC["TE_2_ACN"] = resultDataframeC["TE_2_CN"].apply(
    lambda x: returnLast(x)
)

# Add column for copies not present in clusters
resultDataframeC["TE_1_NCN"] = (
    resultDataframeC["TE_1_ACN"] - resultDataframeC["TE_1_pi"]
)
resultDataframeC["TE_2_NCN"] = (
    resultDataframeC["TE_2_ACN"] - resultDataframeC["TE_2_pi"]
)
resultDataframeC["TE_1_NCN_l"] = np.log(
    resultDataframeC["TE_1_ACN"] - resultDataframeC["TE_1_pi"]
)
resultDataframeC["TE_2_NCN_l"] = np.log(
    resultDataframeC["TE_2_ACN"] - resultDataframeC["TE_2_pi"]
)

# Get ranges for adding colors later
resultDataframeC["TE_1_NCN_R"] = resultDataframeC["TE_1_NCN"].apply(
    lambda x: colorRanges(x)
)
resultDataframeC["TE_2_NCN_R"] = resultDataframeC["TE_2_NCN"].apply(
    lambda x: colorRanges(x)
)

# Some wrangling for plotly go
x = resultDataframeC["ETA"]
y = resultDataframeC["selPen"]
TE1ACN = resultDataframeC["TE_1_ACN"]
TE2ACN = resultDataframeC["TE_2_ACN"]

# Convert count to log scale
resultDataframeC["TE_1_ACN_l"] = np.log(resultDataframeC["TE_1_ACN"])
resultDataframeC["TE_2_ACN_l"] = np.log(resultDataframeC["TE_2_ACN"])

/home/siddharth/mambaforge/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [9]:
resultDataframeC

,State,Generatrion,NTE,AvgCopyNum,CopyNumVar,TEfamilyCN,TEfamilyVR,TEfamilyRg,AvgFit,HGTGen,...,TE_1_ACN,TE_2_ACN,TE_1_NCN,TE_2_NCN,TE_1_NCN_l,TE_2_NCN_l,TE_1_NCN_R,TE_2_NCN_R,TE_1_ACN_l,TE_2_ACN_l
0,FLUX,2501,489026,"[0.2, 0.412, 0.851, 1.487, 2.846, 5.45, 9.57, ...","[0.394, 1.3902560000000002, 3.358799, 6.873831...","{'1': [0.2, 0.412, 0.851, 1.487, 2.846, 5.45, ...","{'1': [[0.394], [1.3902560000000002], [3.35879...","{'1': [0.0, 0.005, 0.015, 0.038, 0.079, 0.16, ...","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,329.215,107.222,323.595,101.656,5.779493,4.621595,>300,100-200,5.796711,4.674901
1,FLUX,2501,363153,"[0.2, 0.416, 0.877, 1.607, 2.948, 5.466, 10.19...","[0.36199999999999993, 1.226944, 3.333871000000...","{'1': [0.2, 0.416, 0.877, 1.607, 2.948, 5.466,...","{'1': [[0.36199999999999993], [1.226944], [3.3...","{'1': [0.0, 0.005, 0.015, 0.038, 0.077, 0.152,...","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,158.722,164.018,152.335,156.732,5.026082,5.054537,100-200,100-200,5.067154,5.099976
2,FLUX,2501,454206,"[0.2, 0.368, 0.752, 1.407, 2.642, 4.725, 8.69,...","[0.3499999999999999, 1.0085760000000001, 2.620...","{'1': [0.2, 0.368, 0.752, 1.407, 2.642, 4.725,...","{'1': [[0.3499999999999999], [1.00857600000000...","{'1': [0.0, 0.004, 0.027, 0.046, 0.069, 0.129,...","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,220.197,223.589,211.511,219.005,5.354277,5.389095,200-300,200-300,5.394523,5.409810
3,FLUX,2501,379290,"[0.2, 0.43, 0.854, 1.618, 3.336, 6.24, 10.709,...","[0.3279999999999999, 1.1551, 3.056683999999999...","{'1': [0.2, 0.43, 0.854, 1.618, 3.336, 6.24, 1...","{'1': [[0.3279999999999999], [1.1551], [3.0566...","{'1': [0.0, 0.004, 0.015, 0.052, 0.092, 0.176,...","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,231.713,150.027,223.905,145.298,5.411222,4.978787,200-300,100-200,5.445500,5.010815
4,FLUX,2501,528252,"[0.2, 0.396, 0.795, 1.581, 3.14, 5.664, 9.681,...","[0.346, 1.0511840000000001, 2.8329749999999994...","{'1': [0.2, 0.396, 0.795, 1.581, 3.14, 5.664, ...","{'1': [[0.346], [1.0511840000000001], [2.83297...","{'1': [0.0, 0.004, 0.012, 0.028, 0.084, 0.179,...","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,145.926,298.558,137.010,293.115,4.920054,5.680565,100-200,200-300,4.983100,5.698964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,FLUX,2501,408587,"[0.2, 0.454, 0.93, 1.836, 3.43, 6.358, 10.868,...","[0.35, 1.333884, 3.7091000000000003, 8.937104,...","{'1': [0.2, 0.454, 0.93, 1.836, 3.43, 6.358, 1...","{'1': [[0.35], [1.333884], [3.7091000000000003...","{'1': [0.0, 0.008, 0.023, 0.038, 0.09, 0.19, 0...","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,222.820,124.081,210.900,121.250,5.351384,4.797855,200-300,100-200,5.406364,4.820935
4996,FLUX,2501,337829,"[0.2, 0.36, 0.7, 1.291, 2.353, 4.349, 8.282, 1...","[0.3739999999999999, 1.0384, 2.512, 5.92631899...","{'1': [0.2, 0.36, 0.7, 1.291, 2.353, 4.349, 8....","{'1': [[0.3739999999999999], [1.0384], [2.512]...","{'1': [0.0, 0.005, 0.012, 0.031, 0.062, 0.125,...","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,301.521,0.000,294.135,0.000,5.684039,-inf,200-300,0,5.708840,-inf
4997,FLUX,2501,624359,"[0.2, 0.482, 0.93, 1.799, 3.361, 6.178, 10.77,...","[0.36199999999999993, 1.435676, 3.737100000000...","{'1': [0.2, 0.482, 0.93, 1.799, 3.361, 6.178, ...","{'1': [[0.36199999999999993], [1.435676], [3.7...","{'1': [0.0, 0.005, 0.016, 0.035, 0.079, 0.162,...","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,278.412,264.030,268.180,258.926,5.591658,5.556542,200-300,200-300,5.629102,5.576063
4998,FLUX,2501,495620,"[0.2, 0.414, 0.815, 1.48, 2.841, 5.211, 9.882,...","[0.3839999999999999, 1.1766040000000002, 3.148...","{'1': [0.2, 0.414, 0.815, 1.48, 2.841, 5.211, ...","{'1': [[0.3839999999999999], [1.17660400000000...","{'1': [0.0, 0.004, 0.016, 0.038, 0.085, 0.151,...","[1.0000000000000007, 1.0000000000000007, 1.000...",50,...,218.441,1